In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import lxml
import requests

In [2]:
url = "https://roscontrol.com/category/produkti/"
url2 = "https://rskrf.ru/ratings/produkty-pitaniya/"
response = requests.get(url)
soup = bs(response.text, "lxml")
categories = [i.text.strip() for i in soup.find_all(class_="catalog__category-wrap-name")]
links = [i.get("href") for i in soup.find_all(class_="catalog__category-item util-hover-shadow")]
all_products = []

In [4]:
for link, category in zip(links, categories):
    response = requests.get(f"https://roscontrol.com{link}")
    soup = bs(response.text, "lxml")
    subcategories = [i.text.strip() for i in soup.find_all(class_="catalog__category-name")]
    links2 = [i.get("href") for i in soup.find_all(class_="catalog__category-item util-hover-shadow")]
    if links2:
        for link2, subcategory in zip(links2, subcategories):
            response = requests.get(f"https://roscontrol.com{link2}")
            soup = bs(response.text, "lxml")
            links3 = [i.get("href") for i in soup.find_all(class_=\
                      "block-product-catalog__item js-activate-rate util-hover-shadow clear")]
            for link3 in links3:
                response = requests.get(f"https://roscontrol.com{link3}")
                soup = bs(response.text, "lxml")
                name = soup.find(class_="main-title testlab-caption-products util-inline-block").text
                if soup.find(class_="total green"):
                    rating = soup.find(class_="total green").text
                    all_ratings = soup.find_all(class_="rate-item group")
                    safety = all_ratings[0].find(class_="rate-item__value").text.strip()
                    quality = all_ratings[-1].find(class_="rate-item__value").text.strip()
                    warning = None
                elif soup.find(class_="total rate blacklist-value"):
                    rating = None
                    safety = None
                    quality = None
                    x = " ".join(soup.find(class_="blacklist__item-danger").text.split())
                    warning = f"В чёрном списке! {x}"
                item = {"name": name, "category": category, "subcategory": subcategory,
                        "safety": safety, "quality": quality, "rating": rating, "warning": warning,
                        "link": f"https://roscontrol.com{link3}"}
                all_products.append(item)
                print(item) # нужно, чтобы видеть, что процесс идёт
    else:
        subcategory = category
        links3 = [i.get("href") for i in \
                  soup.find_all(class_="block-product-catalog__item js-activate-rate util-hover-shadow clear")]
        for link3 in links3:
            response = requests.get(f"https://roscontrol.com{link3}")
            soup = bs(response.text, "lxml")
            name = soup.find(class_="main-title testlab-caption-products util-inline-block").text
            if soup.find(class_="total green"):
                rating = soup.find(class_="total green").text
                all_ratings = soup.find_all(class_="rate-item group")
                safety = all_ratings[0].find(class_="rate-item__value").text.strip()
                quality = all_ratings[-1].find(class_="rate-item__value").text.strip()
                warning = None
            elif soup.find(class_="total rate blacklist-value"):
                rating = None
                safety = None
                quality = None
                x = " ".join(soup.find(class_="blacklist__item-danger").text.split())
                warning = f"В чёрном списке! {x}"
            item = {"name": name, "category": category, "subcategory": subcategory,
                    "safety": safety, "quality": quality, "rating": rating, "warning": warning,
                    "link": f"https://roscontrol.com{link3}"}
            all_products.append(item)
            print(item) # нужно, чтобы видеть, что процесс идёт

{'name': 'Молоко "Экомилк" 3,2% пастеризованное', 'category': 'Молочные продукты', 'subcategory': 'Молоко', 'safety': '95', 'quality': '79', 'rating': '79', 'warning': None, 'link': 'https://roscontrol.com/product/moloko-ekomilk/'}
{'name': 'Молоко "Ашан" 2,5%, пастеризованное', 'category': 'Молочные продукты', 'subcategory': 'Молоко', 'safety': '88', 'quality': '86', 'rating': '76', 'warning': None, 'link': 'https://roscontrol.com/product/ashan-25-pasterizovannoe/'}
{'name': 'Молоко "Вологодское лето" 2,5%, пастеризованное', 'category': 'Молочные продукты', 'subcategory': 'Молоко', 'safety': '83', 'quality': '62', 'rating': '70', 'warning': None, 'link': 'https://roscontrol.com/product/vologodskoe-leto-25-pasterizovannoe/'}
{'name': 'Молоко "Молоко нашей дойки" 2,5%, пастеризованное', 'category': 'Молочные продукты', 'subcategory': 'Молоко', 'safety': '90', 'quality': '70', 'rating': '73', 'warning': None, 'link': 'https://roscontrol.com/product/moloko-nashey-doyki-25-pasterizovannoe/

{'name': 'Кефир "Рузский" 3,2-4%', 'category': 'Молочные продукты', 'subcategory': 'Кефир', 'safety': '95', 'quality': '72', 'rating': '79', 'warning': None, 'link': 'https://roscontrol.com/product/kefir-ruzskiy-32-4/'}
{'name': 'Кефир "Милава"', 'category': 'Молочные продукты', 'subcategory': 'Кефир', 'safety': '95', 'quality': '72', 'rating': '79', 'warning': None, 'link': 'https://roscontrol.com/product/kefir-milava/'}
{'name': 'Творог "Брест-Литовск" 9%', 'category': 'Молочные продукты', 'subcategory': 'Творог', 'safety': '88', 'quality': '74', 'rating': '73', 'warning': None, 'link': 'https://roscontrol.com/product/tvorog-brest-litovsk-9-1/'}
{'name': 'Творог "Савушкин продукт" "Мягкий нежный" 5%', 'category': 'Молочные продукты', 'subcategory': 'Творог', 'safety': '74', 'quality': '30', 'rating': '60', 'warning': None, 'link': 'https://roscontrol.com/product/tvorog-savushkin-produkt-myagkiy-negniy-5/'}
{'name': 'Творог "Останкинское" 0,1%', 'category': 'Молочные продукты', 'subca

{'name': '"Брест-Литовск" 15%', 'category': 'Молочные продукты', 'subcategory': 'Сметана', 'safety': '85', 'quality': '82', 'rating': '74', 'warning': None, 'link': 'https://roscontrol.com/product/smetana-brest-litovsk-15/'}
{'name': 'Масло сливочное "Экомилк" 82,5%', 'category': 'Молочные продукты', 'subcategory': 'Сливочное масло', 'safety': '86', 'quality': '85', 'rating': '74', 'warning': None, 'link': 'https://roscontrol.com/product/maslo-slivochnoie-ekomilk-82-5/'}
{'name': 'Сливочное масло Anchor 82%', 'category': 'Молочные продукты', 'subcategory': 'Сливочное масло', 'safety': '86', 'quality': '80', 'rating': '69', 'warning': None, 'link': 'https://roscontrol.com/product/slivochnoe-maslo-anchor-82-2/'}
{'name': 'Сливочное масло Anchor 82%', 'category': 'Молочные продукты', 'subcategory': 'Сливочное масло', 'safety': '86', 'quality': '80', 'rating': '69', 'warning': None, 'link': 'https://roscontrol.com/product/slivochnoe-maslo-anchor-82-1/'}
{'name': 'Сливочное масло Anchor 82%

{'name': 'Сыр "Киприно" "Российский"', 'category': 'Молочные продукты', 'subcategory': 'Сыры и сырные продукты', 'safety': '76', 'quality': '69', 'rating': '56', 'warning': None, 'link': 'https://roscontrol.com/product/sir-kiprino-rossiyskiy-50/'}
{'name': 'Сыр Valio "Российский"', 'category': 'Молочные продукты', 'subcategory': 'Сыры и сырные продукты', 'safety': '76', 'quality': '64', 'rating': '55', 'warning': None, 'link': 'https://roscontrol.com/product/sir-valio-rossiyskiy-50/'}
{'name': 'Сыр "Белебеевский" "Российский", 50%', 'category': 'Молочные продукты', 'subcategory': 'Сыры и сырные продукты', 'safety': '78', 'quality': '67', 'rating': '57', 'warning': None, 'link': 'https://roscontrol.com/product/belebeevskiy-rossiyskiy/'}
{'name': 'Сыр Moncasa gourmet пармезан', 'category': 'Молочные продукты', 'subcategory': 'Сыры и сырные продукты', 'safety': '77', 'quality': '68', 'rating': '55', 'warning': None, 'link': 'https://roscontrol.com/product/sir-moncasa-gourmet-parmezan/'}
{

{'name': 'Мороженое "Чистая линия" эскимо в апельсиновой глазури', 'category': 'Молочные продукты', 'subcategory': 'Мороженое', 'safety': '65', 'quality': '85', 'rating': '63', 'warning': None, 'link': 'https://roscontrol.com/product/morogenoe-chistaya-liniya-eskimo-v-apelsinovoy-glazuri/'}
{'name': 'Пломбир шоколадный "Чистая линия" в сахарном рожке', 'category': 'Молочные продукты', 'subcategory': 'Мороженое', 'safety': '56', 'quality': '95', 'rating': '68', 'warning': None, 'link': 'https://roscontrol.com/product/plombir-shokoladnyi-chistaia-liniia-v-sakharnom-rozhkie/'}
{'name': 'Пломбир "Чистая линия" "Лакомка Московская" ванильный', 'category': 'Молочные продукты', 'subcategory': 'Мороженое', 'safety': '57', 'quality': '95', 'rating': '72', 'warning': None, 'link': 'https://roscontrol.com/product/lakomka-moskovskaia-plombir-vanil-nyi-vo-vzbitoi-shokoladnoi-ghlazuri-chistaia-liniia/'}
{'name': 'Мороженое пломбир "Айсберри / Филёвский пломбир"', 'category': 'Молочные продукты', 'su

...

{'name': 'Говядина тушеная "Йошкар-Олинский мясокомбинат" высший сорт', 'category': 'Консервы', 'subcategory': 'Мясные консервы', 'safety': '72', 'quality': '54', 'rating': '60', 'warning': None, 'link': 'https://roscontrol.com/product/govyadina-tushenaya-yoshkar-olinskiy-myasokombinat-visshiy-sort/'}
{'name': 'Свинина тушеная "Ремит", высший сорт', 'category': 'Консервы', 'subcategory': 'Мясные консервы', 'safety': '72', 'quality': '54', 'rating': '60', 'warning': None, 'link': 'https://roscontrol.com/product/svinina-tushenaya-remit-visshiy-sort/'}
{'name': 'Свинина тушеная "Мясницкий ряд", высший сорт', 'category': 'Консервы', 'subcategory': 'Мясные консервы', 'safety': '72', 'quality': '54', 'rating': '60', 'warning': None, 'link': 'https://roscontrol.com/product/svinina-tushenaya-myasnitskiy-ryad-visshiy-sort/'}
{'name': 'Говядина тушеная "Сохраним традиции" "Премиум", высший сорт', 'category': 'Консервы', 'subcategory': 'Мясные консервы', 'safety': '72', 'quality': '54', 'rating':

{'name': ' Томаты «Просто!» маринованные', 'category': 'Консервы', 'subcategory': 'Овощные консервы', 'safety': '84', 'quality': '92', 'rating': '69', 'warning': None, 'link': 'https://roscontrol.com/product/tomati-prosto-marinovannie/'}
{'name': ' Томаты «Дядя Ваня» маринованные', 'category': 'Консервы', 'subcategory': 'Овощные консервы', 'safety': '82', 'quality': '96', 'rating': '69', 'warning': None, 'link': 'https://roscontrol.com/product/tomati-dyadya-vanya-marinovannie/'}
{'name': ' Аджика "Богатство вкуса"', 'category': 'Консервы', 'subcategory': 'Овощные консервы', 'safety': '85', 'quality': '79', 'rating': '82', 'warning': None, 'link': 'https://roscontrol.com/product/adgika-bogatstvo-vkusa/'}
{'name': ' Аджика Spar', 'category': 'Консервы', 'subcategory': 'Овощные консервы', 'safety': '82', 'quality': '70', 'rating': '71', 'warning': None, 'link': 'https://roscontrol.com/product/adgika-spar/'}
{'name': ' Аджика "Дядя Ваня"', 'category': 'Консервы', 'subcategory': 'Овощные ко

{'name': 'Бананы "Азбука вкуса"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Свежие плоды и ягоды', 'safety': '86', 'quality': '37', 'rating': '62', 'warning': None, 'link': 'https://roscontrol.com/product/banani-azbuka-vkusa/'}
{'name': 'Бананы "Магнит"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Свежие плоды и ягоды', 'safety': '88', 'quality': '67', 'rating': '78', 'warning': None, 'link': 'https://roscontrol.com/product/banani-magnit/'}
{'name': 'Бананы "Билла"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Свежие плоды и ягоды', 'safety': '87', 'quality': '36', 'rating': '62', 'warning': None, 'link': 'https://roscontrol.com/product/banani-billa/'}
{'name': 'Бананы "Пятерочка"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Свежие плоды и ягоды', 'safety': '87', 'quality': '66', 'rating': '77', 'warning': None, 'link': 'https://roscontrol.com/product/banani-prime-fruit-pyaterochka/'}
{'name': 'Капуста брокколи "О\'Кей"', 'category': 'Фрукты, овощи, 

{'name': 'Огурцы соленые "Фабрика домашних солений"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Квашеные и соленые овощи', 'safety': '69', 'quality': '58', 'rating': '61', 'warning': None, 'link': 'https://roscontrol.com/product/ogurtsi-solenie-fabrika-domashnih-soleniy/'}
{'name': 'Капуста квашеная "Солюшко"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Квашеные и соленые овощи', 'safety': '69', 'quality': '58', 'rating': '61', 'warning': None, 'link': 'https://roscontrol.com/product/kapusta-kvashenaya-solyushka/'}
{'name': 'Капуста квашеная "О\'Кей"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Квашеные и соленые овощи', 'safety': '69', 'quality': '58', 'rating': '61', 'warning': None, 'link': 'https://roscontrol.com/product/kapusta-kvashenaya-okey/'}
{'name': 'Капуста квашеная "Бабушкин погребок"', 'category': 'Фрукты, овощи, грибы', 'subcategory': 'Квашеные и соленые овощи', 'safety': '69', 'quality': '58', 'rating': '61', 'warning': None, 'link': 'https:/

{'name': 'Маргарин "Чудесница" "Универсальный", 55%', 'category': 'Маргарины', 'subcategory': 'Маргарины', 'safety': None, 'quality': None, 'rating': None, 'warning': 'В чёрном списке! Степень нарушения: средняя', 'link': 'https://roscontrol.com/product/margarin-chudesnitsa/'}
{'name': 'Маргарин "Жар-печка", 60%', 'category': 'Маргарины', 'subcategory': 'Маргарины', 'safety': None, 'quality': None, 'rating': None, 'warning': 'В чёрном списке! Степень нарушения: средняя', 'link': 'https://roscontrol.com/product/margarin-gar-pechka/'}
{'name': 'Маргарин "Хозяюшка" "Сливочный Нижегородский", 60%', 'category': 'Маргарины', 'subcategory': 'Маргарины', 'safety': None, 'quality': None, 'rating': None, 'warning': 'В чёрном списке! Степень нарушения: средняя', 'link': 'https://roscontrol.com/product/margarin-hozyayushka/'}
{'name': 'Студень "Снежана"', 'category': 'Готовые блюда и кулинарные изделия', 'subcategory': 'Готовые блюда и кулинарные изделия', 'safety': '28', 'quality': '65', 'rating'

In [5]:
all_products = pd.DataFrame(all_products)
all_products

,name,category,subcategory,safety,quality,rating,warning,link
0,"Молоко ""Экомилк"" 3,2% пастеризованное",Молочные продукты,Молоко,95,79,79,None,https://roscontrol.com/product/moloko-ekomilk/
1,"Молоко ""Ашан"" 2,5%, пастеризованное",Молочные продукты,Молоко,88,86,76,None,https://roscontrol.com/product/ashan-25-paster...
2,"Молоко ""Вологодское лето"" 2,5%, пастеризованное",Молочные продукты,Молоко,83,62,70,None,https://roscontrol.com/product/vologodskoe-let...
3,"Молоко ""Молоко нашей дойки"" 2,5%, пастеризованное",Молочные продукты,Молоко,90,70,73,None,https://roscontrol.com/product/moloko-nashey-d...
4,"Молоко Billa 2,5%, пастеризованное",Молочные продукты,Молоко,89,73,72,None,https://roscontrol.com/product/billa-25-paster...
...,...,...,...,...,...,...,...,...
2890,"Пицца ""Слайс Пицца""",Готовые блюда и кулинарные изделия,Готовые блюда и кулинарные изделия,None,None,None,В чёрном списке! Степень нарушения: средняя,https://roscontrol.com/product/pitstsa-slays-p...
2891,Пицца Pronto,Готовые блюда и кулинарные изделия,Готовые блюда и кулинарные изделия,None,None,None,В чёрном списке! Степень нарушения: средняя,https://roscontrol.com/product/pitstsa-pronto/
2892,Пицца Pizza Hut,Готовые блюда и кулинарные изделия,Готовые блюда и кулинарные изделия,None,None,None,В чёрном списке! Степень нарушения: средняя,https://roscontrol.com/product/pitstsa-pizza-hut/
2893,Пицца Papa John`s,Готовые блюда и кулинарные изделия,Готовые блюда и кулинарные изделия,None,None,None,В чёрном списке! Степень нарушения: средняя,https://roscontrol.com/product/pitstsa-papa-jo...
